In [ ]:
# SQLAlchemy, Sqlite, and Dates

## Setup

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [ ]:
engine = create_engine("sqlite:///../Resources/dow.sqlite", echo=False)

## Explore Database

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Get a list of column names and types
columns = inspector.get_columns('dow')
for c in columns:
    print(c['name'], c["type"])
# columns

In [ ]:
engine.execute('SELECT * FROM dow LIMIT 5').fetchall()

## Reflect and query dates

In [ ]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Dow = Base.classes.dow

In [ ]:
session = Session(engine)

How many dates do we have?

In [ ]:
# Total dates
session.query(func.count(Dow.date)).all()

What is the earliest date?

In [ ]:
# Earliest Date
session.query(Dow.date).order_by(Dow.date).first()

What is the latest date?

In [ ]:
# Latest Date
session.query(Dow.date).order_by(Dow.date.desc()).first()

Find all of the dates great than `2011-03-01`

In [ ]:
session.query(Dow.date).\
    filter(Dow.date > '2011-03-01').\
    order_by(Dow.date).all()

### Important Note! Sqlite does not support a date column type, but SQLAlchemy will allow you to work with dates in the iso format. [sqlite dates](http://docs.sqlalchemy.org/en/latest/dialects/sqlite.html)

# Quick Review of DateTime

In [ ]:
import datetime as dt

In [ ]:
# Print today's date 
print(dt.date.today())
# Print a specific date 
print(dt.date(2017,10 ,8))

In [ ]:
# Print a datetime object 
print(dt.datetime.utcnow())
print(dt.datetime(2017,10 ,8))

Calculate a time delta

In [ ]:
# The date 9 weeks ago from today. 
week_ago = dt.date.today() - dt.timedelta(days=63)
week_ago

In [ ]:
# The number of days between two datetime objects – When the next solar eclipse will be and today.
a = dt.datetime(2023, 10, 14) 
b = dt.datetime.utcnow()
  
# returns a timedelta object 
c = a-b  
print(c)

Query for the Dow closing price 1 week before `2011-04-08` using the datetime library

In [ ]:
# Query for the Dow closing price for `CSCO` 1 week before `2011-04-08` using the datetime library
query_date = dt.date(2011, 4, 8) - dt.timedelta(days=7)
print("Query Date: ", query_date)

In [ ]:
session.query(Dow.date, Dow.close_price).\
    filter(Dow.stock == 'CSCO').\
    filter(Dow.date == query_date).all()

In [ ]:
# Parse out just the day from the datetime object
dt.date.today().strftime("%d")

Putting it all together

In [ ]:
# Query for all stocks, their opening and closing price that match
# following date string in the format `%d`
date_str = "14"
session.query(Dow.date, Dow.stock, Dow.open_price, Dow.close_price).\
    filter(func.strftime("%d", Dow.date) == date_str).all()

## Your Turn!

DataSet Citation: 

Brown, M. S., Pelosi, M. & Dirska, H. (2013). Dynamic-radius Species-conserving Genetic Algorithm for 
the Financial Forecasting of Dow Jones Index Stocks. Machine Learning and Data Mining in Pattern 
Recognition, 7988, 27-41.
